---

Imports

----

In [ ]:
import os
import random
import tempfile,os
from datetime import datetime
from tkinter import Tk, IntVar, filedialog as fd
import customtkinter
from PIL import Image, ImageTk
import cv2
import numpy as np
import matplotlib.pyplot as plt
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib import colors
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms
from torchcam.methods import GradCAM
from pytorch_grad_cam import GradCAMPlusPlus
from pytorch_grad_cam.utils.image import show_cam_on_image


---

Function: Generate heatmap to show the layers attention.

---

Note: I based this heavily on Kaggle Notebooks I found. I did have to alter alot for my use case though.

In [ ]:
def generate_heatmap(model, img_tensor, pred_class, alpha=0.6):
    model.eval()
    activations = []
    gradients = []
    ### Change this layer to an earlied layer, I kept getting a heatmap that would show parts of the image that were not relevant (not where the cancer would be)
    target_layer = model.layer2[-1].conv3

    def forward_hook(module, input, output):
        activations.append(output)

    def backward_hook(module, grad_input, grad_output):
        gradients.append(grad_output[0])

    f_hook = target_layer.register_forward_hook(forward_hook)
    b_hook = target_layer.register_backward_hook(backward_hook)

    output = model(img_tensor)
    score = output[0, pred_class]
    model.zero_grad()
    score.backward()


    act = activations[0].squeeze(0)
    grad = gradients[0].squeeze(0)
    pooled_grad = grad.mean(dim=(1, 2))

    for i in range(act.shape[0]):
        act[i] *= pooled_grad[i]

    heatmap = act.sum(dim=0)
    heatmap = F.relu(heatmap)
    heatmap /= heatmap.max()
    heatmap = heatmap.detach().cpu().numpy()
    heatmap = cv2.resize(heatmap, (img_tensor.shape[3], img_tensor.shape[2]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_TURBO)


    img = img_tensor.squeeze().permute(1, 2, 0).detach().cpu().numpy()
    img = np.uint8(255 * img) if img.max() <= 1.0 else img
    if img.shape[2] == 1:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

    overlay = cv2.addWeighted(img.astype(np.uint8), 1 - alpha, heatmap, alpha, 0)

    f_hook.remove()
    b_hook.remove()

    return overlay

---

Function: Loading model and predicting image class

---

In [ ]:
def predict_image(model_name, image):
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model_path = f"Models/{model_name}.pth"
    model = torch.load(model_path, map_location=device)
    model.eval()
    model.to(device)

    transform = transforms.Compose([
        transforms.Resize(64),
        transforms.CenterCrop(64),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    img_tensor = transform(image).unsqueeze(0).to(device)
    img_tensor.requires_grad_(True)

    with torch.enable_grad():
        predictions = model(img_tensor)  

    predicted_class = torch.argmax(predictions, dim=1)[0]
    confidence_scores = F.softmax(predictions, dim=1)
    confidence_level = confidence_scores[0, predicted_class].item()

    heatmap_img = generate_heatmap(model, img_tensor,predicted_class)
    return predicted_class, confidence_level, heatmap_img

---

Function: Maps model output class number with string equivalent.

---

In [ ]:
def map_outputs(model_name,prediction):
    MRICT = ["Brain Glioma","Brain Meningioma","Brain Pituitary Tumor","Kidney Normal","Kidney Tumor"]
    Hist = ["Breast Benign","Breast Maligmant","Colon Adenocarcinoma","Colon Benign Tissue","Lung Adenocarcinoma","Lung Benign Tissue","Lung Squamous Cell Carcinoma","Lymphoma Chronic Lymphocytic Leukemia","Lymphoma Follicular Lymphoma","Lymphoma Mantle Cell Lymphoma","Oral Normal","Oral Squamous Cell Carcinoma"]
    PapSmear = ["Dyskeratotic","Koilocytotic","Metaplastic","Parabasal","Superficial-Intermediate"]
    BloodSmear = ["Benign","Early Stages","Pre-stage","Advanced"]

    if model_name == "MRI & CT":
        mapped_pred = MRICT[prediction].split()
        
    
    elif model_name == "Histopathology":
        mapped_pred = Hist[prediction].split()

    elif model_name == "Pap Smear":
        mapped_pred = PapSmear[prediction]
        cancer_type = "Cervical"
        variant = mapped_pred
        return cancer_type,variant

    elif model_name == "Blood Smear":
        mapped_pred = BloodSmear[prediction]
        cancer_type = "Acute Lymphoblastic Leukemia"
        variant = mapped_pred
        return cancer_type,variant

    cancer_type = mapped_pred[0]
    variant_list = mapped_pred[1:]
    variant = " ".join(variant_list)
    return cancer_type,variant


---

GUI Class

---

In [ ]:
class P2P_GUI:

    def __init__(self, root):
        self.root = root
        self.root.title("Pixels2Prognosis")
        self.root.geometry("660x360")
        self.root.resizable(False, False)

        self.uploaded_image = None
        self.model_var = IntVar()

        ###
        ### Setup GUI size with left and right frames for input and output
        ###


        left_width = 470
        right_width = 190
        height = 360


        ###
        ### Left Frame Creation
        ###

        self.left_frame = customtkinter.CTkFrame(
            master=root,
            width=left_width,
            height=height,
            fg_color="#000000"
        )
        self.left_frame.place(x=0, y=0)


        ###
        ### Right Frame Creation
        ###
        

        self.right_frame = customtkinter.CTkFrame(
            master=root,
            width=right_width,
            height=height,
            fg_color="#E6E7E2"
        )
        
        self.right_frame.place(x=left_width, y=0)



        ###
        ### Left Frame Widgets
        ###



        ### Image Display


        self.image_label = customtkinter.CTkLabel(
            master=self.left_frame,
            text="No Image",
            font=("Arial", 14),
            text_color="#ffffff",
            height=200,
            width=200,
            corner_radius=17,
            bg_color="#ffffff",
            fg_color="#000000",
        )
        self.image_label.place(x=20, y=50)



        ###
        ### Upload Button
        ###


        self.upload_button = customtkinter.CTkButton(
            master=self.left_frame,
            text="Upload Image",
            font=("Arial", 14),
            text_color="#E6E7E2",
            hover_color="#949494",
            height=30,
            width=110,
            border_width=2,
            corner_radius=6,
            border_color="#ffffff",
            bg_color="#000000",
            fg_color="#ff7a7a",
            command=self.upload_image
        )
        self.upload_button.place(x=70, y=290)



        ###
        ### Title Text for Choose Scan type
        ###


        self.model_prompt_label = customtkinter.CTkLabel(
            master=self.left_frame,
            text="Choose a scan:",
            font=("Arial", 14),
            text_color="#ffffff",
            bg_color="#000000"
        )
        self.model_prompt_label.place(x=270, y=105)



        ###
        ### Loop to create radio buttons for each scan type
        ###


        self.model_options = [
            "MRI & CT","Pap Smear","Blood Smear","Histopathology"
        ]

        for idx, model_name in enumerate(self.model_options):
            radio = customtkinter.CTkRadioButton(
                master=self.left_frame,
                variable=self.model_var,
                value=idx,
                text=model_name,
                text_color="#ffffff",
                border_color="#E6E7E2",
                fg_color="white",
                hover_color="#2F2F2F",
            )
            radio.place(x=270, y=140 + idx * 30)



        ###
        ### Patient ID Entry
        ###


        self.text_entry = customtkinter.CTkEntry(
            master=self.left_frame,
            width=150,
            height=30,
            placeholder_text="Patient ID #...",
            font=("Arial", 12),
            text_color="#000000",
            fg_color="#ffffff",
            bg_color="#ffffff",
            corner_radius=6
        )
        self.text_entry.place(x=270, y=45)


        ###
        ### Predict Button
        ###


        self.predict_button = customtkinter.CTkButton(
            master=self.left_frame,
            text="Predict",
            font=("Arial", 14),
            text_color="#E6E7E2",
            hover_color="#949494",
            height=30,
            width=95,
            border_width=2,
            corner_radius=6,
            border_color="#ffffff",
            bg_color="#000000",
            fg_color="#ff7a7a",
            command=self.process_image
        )
        self.predict_button.place(x=275, y=290)



        ###
        ### Save PDF Button
        ###


        self.save_pdf_button = customtkinter.CTkButton(
            master=self.right_frame,
            text="Save as PDF",
            font=("Arial", 14),
            text_color="#E6E7E2",
            hover_color="#949494",
            height=30,
            width=95,
            border_width=2,
            corner_radius=6,
            border_color="#ffffff",
            fg_color="#ff7a7a",
            command=self.save_report_pdf
        )
        self.save_pdf_button.place(x=45, y=290)



        ###
        ### Output Labels - Right Frame
        ###



        ###
        ### Precition
        ###

        self.output_box1 = customtkinter.CTkLabel(
            master=self.right_frame,
            text="Prediction:",
            font=("Arial", 12),
            text_color="#000000",
            height=30,
            width=170,
            corner_radius=0,
            bg_color="#ffffff",
            fg_color="#ff7a7a",
        )
        self.output_box1.place(x=10, y=120)



        ###
        ### Confidence
        ###



        self.output_box2 = customtkinter.CTkLabel(
            master=self.right_frame,
            text="Confidence:",
            font=("Arial", 12),
            text_color="#000000",
            height=30,
            width=170,
            corner_radius=0,
            bg_color="#ffffff",
            fg_color="#ff7a7a",
        )
        self.output_box2.place(x=10, y=220)



        ###
        ### Variant
        ###



        self.output_box3 = customtkinter.CTkLabel(
            master=self.right_frame,
            text="Variant:",
            font=("Arial", 12),
            text_color="#000000",
            height=30,
            width=170,
            corner_radius=0,
            bg_color="#ffffff",
            fg_color="#ff7a7a",
        )
        self.output_box3.place(x=10, y=170)




        ###
        ### Logo Image
        ###



        bottom_left_img = Image.open("IMG_1485.png")
        bottom_left_img = bottom_left_img.resize((100, 100))
        self.bottom_left_img_tk = ImageTk.PhotoImage(bottom_left_img)

        self.bottom_left_image_label = customtkinter.CTkLabel(
            master=self.right_frame,
            image=self.bottom_left_img_tk,
            text="",
            bg_color="#000000"
        )
        self.bottom_left_image_label.place(x=45, y=10)



    ###
    ###  Funtion: Upload Image
    ###


    def upload_image(self):
        file_path = fd.askopenfilename(filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
        if file_path:
            img = Image.open(file_path)
            img_resized = img.resize((200, 200))
            self.original_image = img
            self.uploaded_image = ImageTk.PhotoImage(img_resized)
            self.image_label.configure(image=self.uploaded_image, text="")


    ###
    ###Function: Process image, Make Prediction on Image, Generate Heatmap and output results.
    ###


    def process_image(self):
        selected_model = self.model_var.get()
        if self.uploaded_image:
            model_name = self.model_options[selected_model] if 0 <= selected_model < len(self.model_options) else "Unknown"
            predicted_class,confidence,heatmap = predict_image(model_name,self.original_image)
            cancer_type,variant = map_outputs(model_name,predicted_class)
            confidence_percentage = confidence *100
            self.output_box1.configure(text=f"{cancer_type}")
            self.output_box2.configure(text=f"Confidence: {confidence_percentage:.2f}%")
            self.output_box3.configure(text=f"{variant}")
            self.heatmap = heatmap



    ###
    ###Function: Save Report as PDF
    ###


    def save_report_pdf(self):
        if not self.uploaded_image:
            return

        patient_id = self.text_entry.get()
        now = datetime.now()
        date_str = now.strftime("%Y-%m-%d")
        time_str = now.strftime("%H:%M:%S")
        prediction = self.output_box1.cget("text")
        confidence = self.output_box2.cget("text").replace("Confidence: ", "").strip()
        variant = self.output_box3.cget("text").replace("Variant: ", "").strip()

        heatmap_img = Image.fromarray(self.heatmap)
        temp_image_path = os.path.join(tempfile.gettempdir(), "temp_image.jpg")
        heatmap_path = os.path.join(tempfile.gettempdir(), "temp_heatmap.jpg")
        self.original_image.save(temp_image_path)
        heatmap_img.save(heatmap_path)

        default_filename = f"report_{date_str}_{patient_id}.pdf"
        save_path = fd.asksaveasfilename(
            defaultextension=".pdf",
            initialfile=default_filename,
            filetypes=[("PDF Files", "*.pdf")]
        )
        if not save_path:
            return

        c = canvas.Canvas(save_path, pagesize=letter)
        width, height = letter
        margin = 50

        c.setFont("Helvetica-Bold", 18)
        c.setFillColor(colors.HexColor("#2C3E50"))
        c.drawString(margin, height - 60, "■ Cancer Detection Report")

        c.setFont("Helvetica", 11)
        c.setFillColor(colors.black)
        c.drawString(margin, height - 80, f"Patient ID: {patient_id}")
        c.drawString(margin, height - 95, f"Date: {date_str}    |    Time: {time_str}")

        c.setStrokeColor(colors.lightgrey)
        c.setLineWidth(1)
        c.line(margin, height - 105, width - margin, height - 105)

        c.setFillColor(colors.HexColor("#F4F4F4"))
        c.rect(margin, height - 180, width - 2 * margin, 60, fill=1, stroke=0)

        c.setFillColor(colors.black)
        c.setFont("Helvetica-Bold", 12)
        c.drawString(margin + 10, height - 160, "Diagnosis Summary:")
        c.setFont("Helvetica", 11)
        c.drawString(margin + 10, height - 175, f"Cancer Type: {prediction}")
        c.drawString(margin + 250, height - 175, f"Variant: {variant}")
        c.drawString(margin + 400, height - 175, f"Confidence: {confidence}")

        img_width = 220
        img_height = 220
        spacing = 50
        img_y = height - 420

        orig_x = margin
        heat_x = margin + img_width + spacing

        c.setStrokeColor(colors.HexColor("#AAAAAA"))
        c.rect(orig_x - 2, img_y - 2, img_width + 4, img_height + 4, stroke=1, fill=0)
        c.drawImage(temp_image_path, orig_x, img_y, width=img_width, height=img_height)

        c.rect(heat_x - 2, img_y - 2, img_width + 4, img_height + 4, stroke=1, fill=0)
        c.drawImage(heatmap_path, heat_x, img_y, width=img_width, height=img_height)

        c.setFont("Helvetica", 10)
        c.drawCentredString(orig_x + img_width // 2, img_y - 15, "Original Scan")
        c.drawCentredString(heat_x + img_width // 2, img_y - 15, "Heatmap Overlay")

        c.save()





if __name__ == "__main__":
    customtkinter.set_appearance_mode("dark")
    root = Tk()
    app = P2P_GUI(root)
    root.mainloop()


C:\Users\ckell\AppData\Local\Temp\ipykernel_19496\1221206996.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path, map_location=device)
